In [1]:
import os

In [2]:
%pwd

'e:\\techfolders\\Text-Summarizer-Project\\research'

In [3]:
#we are in research folder so we need to go back to root folder and perfom taks from there
os.chdir('../')

In [4]:
%pwd

'e:\\techfolders\\Text-Summarizer-Project'

In [5]:
# 1. updating config.yaml file
# 2. Updating params.py done while modular coding, not now


In [6]:
# 3. update entity ....entity is return type of a function
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    #Defining the return type of variables in config.yaml updated earlier
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [7]:
# 4. Update configuration manager in src/config folder
from textsummarizer.constants import *   #import all the constants using * from my textsummarizer folder and its constant folder
from textsummarizer.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)   #config_filepath is in constants/__init__.py file and this variable has the path to config file which
        #has to be read and stored in self.config
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) #creating directories using the function written in common.py file. self.config has the path
        #to the config.yaml file and this has the artifacts_root variable. So whatever is stored in that variable the directory will be created according
        # to that. Funnily in simplest form it can be written as 'create_directories('artifacts')'   !!
    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion    #taking the configuration for the data_ingestion part only

        create_directories([config.root_dir])   #creating directory for data ingestionp part in the artifacts folder

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

In [24]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        print('zip file path',self.config.local_data_file)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [26]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-19 23:02:50,550: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-19 23:02:50,553: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-19 23:02:50,556: INFO: common: created directory at: artifacts]
[2024-11-19 23:02:50,559: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-19 23:02:53,347: INFO: 1393347048: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FD73:244DD8:5586E:620CF:673CC9C8
Accept-Ranges: bytes
Date: Tue, 19 Nov 2024 17:32:51 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4739-BOM
X-Cache: HIT
